<a href="https://colab.research.google.com/github/Wahiba275/PySpark_RealEstate_Analysis/blob/main/PySpark_RealEstate_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=6f460dfaffeb07ab03d7aeafedf3263a22ea8655c4f42284f6dd4d51a6ea08ed
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.appName("TP PySpark").master("local[*]").getOrCreate()

In [4]:
dfAchats=spark.read.json("achats.json")

In [5]:
dfAchats=spark.read.option("multiline",True).json("achats.json")

In [6]:
dfAchats.show()


+-----------------+---+---------+------+----+------+------+-------+-----------+--------------+----------+
|          adresse| id|nb_pieces|   nom|numC|prenom|  prix|surface|        tel|          type|     ville|
+-----------------+---+---------+------+----+------+------+-------+-----------+--------------+----------+
|      agdal rabat|  1|        4| ahmed|1111| krami|900000|    110| 0633445530| haut-standing|     rabat|
|      agdal rabat|  2|        3| ahmed|1111| krami|600000|     80| 0633445530|moyen-standing|     rabat|
|maarif casablanca|  3|        4|rachid|2222| Ahmed|900000|    110| 0754352130| haut-standing|casablanca|
|maarif casablanca|  4|        4| hajar|3333|  said|900000|    110|06938473230| haut-standing|casablanca|
|maarif casablanca|  5|        4|  said|4444|belhaj|900000|    110|06938473430| haut-standing|casablanca|
|      agdal rabat|  6|        4| hajar|5555|  said|900000|    110|06938473230|moyen-standing|     rabat|
|      agdal rabat|  7|        3| hajar|5555| 

In [7]:
dfAchats.printSchema()

root
 |-- adresse: string (nullable = true)
 |-- id: long (nullable = true)
 |-- nb_pieces: long (nullable = true)
 |-- nom: string (nullable = true)
 |-- numC: long (nullable = true)
 |-- prenom: string (nullable = true)
 |-- prix: long (nullable = true)
 |-- surface: long (nullable = true)
 |-- tel: string (nullable = true)
 |-- type: string (nullable = true)
 |-- ville: string (nullable = true)



#  Display the first two cities where the company has achieved higher sales (in terms of revenues)






In [8]:
from pyspark.sql import functions as F

In [9]:
total_revenues_per_city = dfAchats.groupBy("ville").agg(F.sum("prix"))
total_revenues_per_city.show()

+----------+---------+
|     ville|sum(prix)|
+----------+---------+
|casablanca|  2700000|
|     rabat|  3000000|
+----------+---------+



In [10]:
top_cities=total_revenues_per_city.orderBy(F.desc("sum(prix)")).limit(2)
top_cities.show()

+----------+---------+
|     ville|sum(prix)|
+----------+---------+
|     rabat|  3000000|
|casablanca|  2700000|
+----------+---------+



# Display the list of clients who own more than one apartment in Rabat

In [14]:
rabat_data = dfAchats.filter(dfAchats.ville=="rabat").groupBy("nom").count()
top_clients = rabat_data.filter(rabat_data['count'] > 1)
top_clients.show()


+-----+-----+
|  nom|count|
+-----+-----+
|hajar|    2|
|ahmed|    2|
+-----+-----+



# Display the number of apartments sold in Casablanca

In [17]:
casablanca_apartments_count = dfAchats.filter(dfAchats.ville == 'casablanca').count()
casablanca_apartments_count

3